In [4]:
import pandas as pd
import mysql.connector
from mysql.connector import errorcode
import numpy as np

# Conectar a MySQL
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='XXXXXXX'
)

cursor = conn.cursor()

# Borrar la base de datos yelp si existe y volverla a crear
cursor.execute('DROP DATABASE IF EXISTS Google_Yelp')
cursor.execute('CREATE DATABASE Google_Yelp')
cursor.execute('USE Google_Yelp')

# Crear tablas de dimensiones
cursor.execute('''
    CREATE TABLE IF NOT EXISTS dim_business (
        business_id VARCHAR(255) PRIMARY KEY,
        name VARCHAR(255),
        address VARCHAR(255),
        city VARCHAR(255),
        state VARCHAR(255),
        postal_code VARCHAR(20),
        categories TEXT
    )
''')
cursor.execute('''
    CREATE TABLE IF NOT EXISTS dim_user (
        user_id VARCHAR(255) PRIMARY KEY,
        name VARCHAR(255)
    )
''')
cursor.execute('''
    CREATE TABLE IF NOT EXISTS dim_review (
        review_id VARCHAR(255) PRIMARY KEY,
        text TEXT,
        date DATETIME
    )
''')

# Crear tablas de hechos
cursor.execute('''
    CREATE TABLE IF NOT EXISTS fact_business (
        business_id VARCHAR(255),
        latitude DOUBLE,
        longitude DOUBLE,
        stars DOUBLE,
        review_count INT,
        is_open INT,
        FOREIGN KEY (business_id) REFERENCES dim_business(business_id)
    )
''')
cursor.execute('''
    CREATE TABLE IF NOT EXISTS fact_checkin (
        business_id VARCHAR(255),
        date DATETIME,
        FOREIGN KEY (business_id) REFERENCES dim_business(business_id)
    )
''')
cursor.execute('''
    CREATE TABLE IF NOT EXISTS fact_review (
        review_id VARCHAR(255),
        user_id VARCHAR(255),
        business_id VARCHAR(255),
        stars DOUBLE,
        FOREIGN KEY (review_id) REFERENCES dim_review(review_id),
        FOREIGN KEY (user_id) REFERENCES dim_user(user_id),
        FOREIGN KEY (business_id) REFERENCES dim_business(business_id)
    )
''')
cursor.execute('''
    CREATE TABLE IF NOT EXISTS fact_tip (
        user_id VARCHAR(255),
        business_id VARCHAR(255),
        text TEXT,
        date DATETIME,
        compliment_count INT,
        FOREIGN KEY (user_id) REFERENCES dim_user(user_id),
        FOREIGN KEY (business_id) REFERENCES dim_business(business_id)
    )
''')
cursor.execute('''
    CREATE TABLE IF NOT EXISTS fact_user (
        user_id VARCHAR(255),
        review_count INT,
        FOREIGN KEY (user_id) REFERENCES dim_user(user_id)
    )
''')

# Crear tablas adicionales para Google Maps
cursor.execute("""
CREATE TABLE IF NOT EXISTS DimLugarGMaps (
    id INT AUTO_INCREMENT PRIMARY KEY,
    gmap_id VARCHAR(255) UNIQUE,
    name VARCHAR(255),
    address VARCHAR(255),
    city VARCHAR(255),
    state VARCHAR(255),
    zip_code VARCHAR(255),
    latitude DECIMAL(10, 8),
    longitude DECIMAL(11, 8),
    category VARCHAR(255)
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS FactReviewGMaps (
    id INT AUTO_INCREMENT PRIMARY KEY,
    gmap_id VARCHAR(255),
    user_name VARCHAR(255),
    rating INT,
    text TEXT,
    FOREIGN KEY (gmap_id) REFERENCES DimLugarGMaps(gmap_id)
)
""")
# Crear tabla para relacionar fact_business y DimLugarGMaps basándose en latitud y longitud
cursor.execute('''
    CREATE TABLE IF NOT EXISTS fact_business_gmaps (
        business_id VARCHAR(255),
        gmap_id VARCHAR(255),
        latitude DOUBLE,
        longitude DOUBLE,
        FOREIGN KEY (business_id) REFERENCES fact_business(business_id),
        FOREIGN KEY (gmap_id) REFERENCES DimLugarGMaps(gmap_id)
    )
''')

# Cerrar la conexión
cursor.close()
conn.close()

print("Tablas creadas correctamente en la base de datos MySQL.")


Tablas creadas correctamente en la base de datos MySQL.


In [ ]:
# Función para insertar datos desde un DataFrame
def insert_data(df, table_name):
    placeholders = ', '.join(['%s'] * len(df.columns))
    columns = ', '.join(df.columns)
    sql = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"
    cursor.executemany(sql, df.values.tolist())
    conn.commit()

In [ ]:
# Cargar los datos transformados
dim_business = pd.read_csv('dim_business.csv')
dim_user = pd.read_csv('dim_user.csv')
dim_review = pd.read_csv('dim_review.csv')
fact_business = pd.read_csv('fact_business.csv')
fact_checkin = pd.read_csv('fact_checkin.csv')
fact_review = pd.read_csv('fact_review.csv')
fact_tip = pd.read_csv('fact_tip.csv')
fact_user = pd.read_csv('fact_user.csv')

In [ ]:
# Insertar datos en las tablas de dimensiones
insert_data(dim_business, 'dim_business')
insert_data(dim_user, 'dim_user')
insert_data(dim_review, 'dim_review')

In [ ]:
# Insertar datos en las tablas de hechos
insert_data(fact_business, 'fact_business')
insert_data(fact_checkin, 'fact_checkin')
insert_data(fact_review, 'fact_review')
insert_data(fact_tip, 'fact_tip')
insert_data(fact_user, 'fact_user')

In [ ]:

# Cerrar la conexión
cursor.close()
conn.close()




In [ ]:
print("Datos insertados correctamente en la base de datos MySQL.")